Define some variables and input files

In [ ]:
import os

data_path = "../../../data/contours/svg"
svg_file = "shapes.svg"
# svg_file = "transform.svg"

#data_path = "."
#svg_file = "ellipse.svg"


input_file = os.path.join(data_path, svg_file)
# print(os.path.abspath(input_file))
# print(os.listdir(data_path))

output_file = "drawing.mesh"

Load SVG file using `svgpathtools`

In [ ]:
from svgpathtools import Document, Path, Line, QuadraticBezier, CubicBezier, Arc, is_bezier_path, is_path_segment, svg2paths, wsvg

doc = Document(input_file)
paths = doc.paths()

Some utility functions

In [ ]:
def lerp(a,b,t):
    return (1-t)*a+t*b

def line_to_cubic(line : Line):
    p_0,p_3 = line.bpoints()
    return (CubicBezier(p_0, lerp(p_0, p_3, 1/3), lerp(p_0, p_3, 2/3), p_3), [1,1,1,1])

def quadratic_to_cubic(quad : QuadraticBezier):
    q_0,q_1,q_2 = quad.bpoints()
    return (CubicBezier(q_0, lerp(q_0,q_1, 2/3), lerp(q_1,q_2, 1/3), q_2), [1,1,1,1])

def arc_to_cubic(arc: Arc):
    # Note: This assumes we have the full arc; will need to be adapted if we only have
    # a portion of the arc

    # Note -- the major arc radius is already encoded into arc.start/arc.end
    q_0 = arc.start
    q_1 = arc.end

    # Note: The shoulder point of the ellipse is at the intersection of the segments 
    # from Bezier control points B0 to B2 and from B1 to B3; 
    # hence the factor of 2 applied to the minor arc
    sc = -2*arc.radius.imag if arc.theta == 180 else 2*arc.radius.imag
    t_0 = complex(0, sc) 
    t_1 = complex(0, -sc)

    return (CubicBezier(q_0, q_0 + t_0, q_1 - t_1, q_1), [3,1,1,3])

    # for a in arc.as_cubic_curves(1):
    #     return a

def segment_as_cubic(seg):
    if isinstance(seg,Line):
        return line_to_cubic(seg)
    elif isinstance(seg,QuadraticBezier):
        return quadratic_to_cubic(seg)
    elif isinstance(seg, CubicBezier):
        return (seg, [1,1,1,1])
    elif isinstance(seg,Arc):
        return arc_to_cubic(seg)
    else:
        raise Exception(f"'{type(seg)}' type not supported yet")


Print out the paths and segments

In [ ]:
for p_idx, p in enumerate(paths):
    for seg_idx, seg in enumerate(p):
        try:
            if is_bezier_path(p):
                cubic,weights =  segment_as_cubic(seg)
                print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}\n\tas cubic: {cubic}")
            elif isinstance(seg, Arc):
                print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}; theta: {seg.theta}; delta: {seg.delta}; phi: {seg.phi}")

                print(f"""  [Arc: start: {seg.start}, end {seg.end}]""")

        except Exception as err:
            # print(f"[Path {p_idx}; Seg {seg_idx}]:\n\t{seg}")
            print(f"parsed unsupported type {type(seg)}. Msg={err}")


Convert paths to mfem mesh of cubic Bezier segments using ASCII output

In [ ]:
import numpy as np

# Create an mfem mesh

header = """
MFEM NURBS mesh v1.0

# MFEM Geometry Types (see fem/geom.hpp):
#
# SEGMENT = 1 | SQUARE = 3 | CUBE = 5
#
# element: <attr> 1 <v0> <v1>
# edge: <idx++> 0 1  <-- idx increases by one each time
# knotvector: <order> <num_ctrl_pts> [knots]; sizeof(knots) is 1+order+num_ctrl_pts
# weights: array of weights corresponding to the NURBS element
# FES: list of control points; vertex control points at top, then interior control points

dimension
1
"""

elem_cnt = 0
vert_cnt = 0

elems = []
edges = []
knots = []

# mfem format lists the endpoints and then the interiors
wgts_ends = []
wgts_ints = []
dof_ends = []
dof_ints = []

print(paths)

for p_idx, p in enumerate(paths):

    if not all(map(is_path_segment, p)):
        continue

    for seg_idx, seg in enumerate(p):
        cubic, weights = segment_as_cubic(seg)
        # print(f"Cubic: {cubic}")

        elems.append(" ".join(map(str,[p_idx + 1, 1, vert_cnt, vert_cnt + 1])))

        edges.append(f"{elem_cnt} 0 1")

        # Hack -- assume for now that the order is always 3 and weights are always 1
        knots.append("3 4 0 0 0 0 1 1 1 1")
        wgts_ends.append(f"{weights[0]} {weights[3]}")
        wgts_ints.append(f"{weights[1]} {weights[2]}")

        dof_ends.append(" ".join(map(str,[cubic.start.real, cubic.start.imag])))
        dof_ends.append(" ".join(map(str,[cubic.end.real, cubic.end.imag])))
        dof_ints.append(" ".join(map(str,[cubic.control2.real, cubic.control2.imag])))
        dof_ints.append(" ".join(map(str,[cubic.control1.real, cubic.control1.imag])))

        vert_cnt += 2
        elem_cnt += 1 

mfem_file = []
mfem_file.append(header)
mfem_file.append("""
elements
{}
{}
""".format(elem_cnt, "\n".join(elems)))

mfem_file.append("""
boundary
0
""")

mfem_file.append("""
edges
{}
{}
""".format(elem_cnt, "\n".join(edges)))

mfem_file.append(f"""
vertices
{vert_cnt}
""")

mfem_file.append("""
knotvectors
{}
{}
""".format(elem_cnt, "\n".join(knots)))

mfem_file.append("""
weights
{}
{}
""".format("\n".join(wgts_ends), "\n".join(wgts_ints)))

mfem_file.append("""
FiniteElementSpace
FiniteElementCollection: NURBS
VDim: 2
Ordering: 1

{}
{}
""".format("\n".join(dof_ends),"\n".join(dof_ints)))

with open(output_file, mode='w') as f:
    f.write("\n".join(mfem_file))
    print(f"wrote '{output_file}' with {vert_cnt} vertices and {elem_cnt} elements")
